-- this notebook aim to process the DRUG and CCL information into vector feature, can IC50 value into edge feature, then train the model GNN to get primary result --

1) Drug Featurization (SELFIES → SELFIES -- RDKit --> Molecular Graph)

In [10]:
import pandas as pd
import numpy as np

In [14]:
# Load datasets
ic50_df = pd.read_csv("5_dataset/final_IC50_interaction.csv")
expression_df = pd.read_csv("5_dataset/final_lung_expression_data.csv", index_col=0)
mutation_df = pd.read_csv("5_dataset/final_lung_mutation_data.csv")



C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1520\3324339672.py:4: DtypeWarning: Columns (4,5,6,10,11,12,16,17,21,23,25,26,27,30,31,32,33,34,35,36,37,39,40,41,44,46,47,49,53,61,64,66,68,69,70,71,73,74,75,76,79,81,82,83,84,85,86,87,88,89,93,94,97,98,100,101,102,103,104,106,108,109,110,111,113,114,115,116,117,120,121,125,126,128,129,131,133,134,136,139,140,141,142,149,150,155,156,159,162,166,170,171,173,175,177,178,179,180,183,185,187,193,194,197,198,199,200,203,205,206,212,214,215,216,217,218,219,220,223,224,225,230,231,232,235,236,237,238,240,242,243,244,245,246,247,248,250,253,255,256,259,260,261,262,264,265,269,273,277,278,279,282,285,286,287,288,289,294,295,296,299,300,304,305,306,308,313,318,319,322,323,324,325,326,327,328,329,330,331,334,336,337,339,340,341,342,343,345,350,351,353,357,360,362,363,364,367,368,369,370,371,373,374,375,376,377,378,382,383,384,386,389,390,391,395,397,401,405,406,409,410,412,413,414,415,416,417,418,419,422,423,426,427,428,429,430,431,432,438,439,441,442

In [18]:
#drop column 'Unnamed:0' and 'cell line name' in the mutation_df 
mutation_df = mutation_df.drop(columns=['Unnamed: 0', 'cell line name'])
#set 'Depmap Id' as index
mutation_df = mutation_df.set_index('Depmap Id')
mutation_df.head(5)


,LOC100288069,LINC01128,LINC01128.1,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,PERM1,HES4,...,VBP1,VBP1.1,SPRY3,TMLHE,SPRY3.1,VAMP7,IL9R,DDX11L16,CD24,CD24.1
Depmap Id,,,,,,,,,,,,,,,,,,,,,
ACH-000698,0,0,0,0.24,0.00044,0.01222,0.01244,0.00132,0.08106,0.00198,...,0.98571,0.98571,0,0,0.10967,0.0,0.8609,NA,0.00515,0
ACH-000523,0,0,0,0.60134,0.00330,0.18282,0.20615,0.54334,0.04282,0.01493,...,0.93083,0.93083,0,0,0.0485,0.0,0.96913,NA,0.0,0
ACH-000610,0,0,0,0.46804,0.00497,0.10173,0.10419,0.67507,0.71297,0.01195,...,0.89152,0.89152,0,NA,0,0.0,0.81503,0.7417,0.02698,0.0214
ACH-000774,0,0,0,0.81141,0.04441,0.07674,0.07868,0.00748,0.63506,0.01121,...,0.9039,0.9039,0,0,0.09443,0.0,0.75237,NA,0.11971,0.18987
ACH-000875,0,0.01477,0.01477,0.10924,0.03631,0.72194,0.51957,0.0,0.56722,0.00419,...,0.6679,0.6679,0.9,0.47037,0.26403,0.26355,0.4568,0.98017,0.0,0


In [20]:
#rename CCL/Gene to 'Depmap ID' in expression_df
expression_df = expression_df.rename(columns={'CCL/Gene': 'Depmap Id'})
#set 'Depmap Id' as index
expression_df = expression_df.set_index('Depmap Id')

In [21]:
expression_df.head(5)

,TSPAN6 (7105),TNMD (64102),DPM1 (8813),SCYL3 (57147),C1orf112 (55732),FGR (2268),CFH (3075),FUCA2 (2519),GCLC (2729),NFYA (4800),...,ZNF772 (400720),ZNF257 (113835),PCNX3 (399909),ADAM32 (203102),ACSL5 (51703),LRRC8B (23507),ABCB8 (11194),H3C12 (8356),SND1 (27044),ZNF785 (146540)
Depmap Id,,,,,,,,,,,,,,,,,,,,,
ACH-000327,3.337711,0.000000,5.927659,1.944858,2.678072,0.014355,3.090853,6.011451,3.642702,2.879706,...,2.435629,0.042644,3.727920,0.070389,0.275007,1.903038,4.927896,0.475085,6.916238,2.405992
ACH-000705,4.412104,0.000000,6.412782,2.367371,4.275752,0.042644,0.201634,2.563158,6.104546,4.585563,...,2.381283,0.084064,5.078097,0.970854,0.084064,2.765535,4.148121,0.604071,6.446256,1.321928
ACH-000528,4.512227,0.000000,7.099926,2.843984,4.672991,0.014355,0.815575,6.709153,3.982765,4.236493,...,1.622930,0.000000,4.856488,0.321928,0.176323,3.367371,4.921246,0.378512,6.367546,2.568032
ACH-000800,3.135863,0.000000,6.668318,2.592158,4.348374,0.014355,0.422233,5.124742,4.712596,5.424922,...,2.990955,0.201634,4.495695,0.137504,0.189034,3.978196,4.331992,0.176323,6.510804,3.699330
ACH-000176,1.111031,0.042644,7.071677,2.007196,3.586164,0.014355,0.536053,7.026911,4.141596,3.987321,...,2.100978,2.906891,4.650765,1.063503,2.782409,0.985500,5.785289,0.422233,7.630959,3.212569


In [ ]:
# Align cell line IDs
cell_line_ids = list(set(ic50_df['Depmap Id']) & set(expression_df.index) & set(mutation_df.index)) #intersection of CCL
expression_df = expression_df.loc[cell_line_ids] #loc keep only what aligns CCL to cell_line_ids
mutation_df = mutation_df.loc[cell_line_ids]

In [24]:
expression_df.shape, mutation_df.shape, ic50_df.shape   

((50, 16383), (50, 20192), (12933, 19))

In [32]:
mutation_df.head(10)
# it have Na values !!!

,LOC100288069,LINC01128,LINC01128.1,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,PERM1,HES4,...,VBP1,VBP1.1,SPRY3,TMLHE,SPRY3.1,VAMP7,IL9R,DDX11L16,CD24,CD24.1
Depmap Id,,,,,,,,,,,,,,,,,,,,,
ACH-000766,0.0054,0,0,0.2199,0.00319,0.56622,0.45567,0.15456,0.6955,0.00582,...,0.93662,0.93662,0.0152,0,0.193,0.21055,0.72042,1,0.01729,0.03054
ACH-000638,0.00714,0.00524,0.00524,0.63022,0.68687,0.76282,0.63827,0.00266,0.61781,0.00116,...,0.91177,0.91177,0.9595,0.83335,0.2278,0.07095,0.54917,NA,0.0238,0.0357
ACH-000563,0.04862,0.05165,0.05165,0.7013,0.03529,0.65608,0.57020,0.15066,0.80167,0.0188,...,1.0,1.0,0.9677,NA,0.32907,0.0,0.64642,NA,0.13725,0.18983
ACH-000781,0,0.00303,0.00303,0.55523,0.06030,0.03324,0.02860,0.01503,0.43294,0.0063,...,0.94569,0.94569,0,NA,0,0.0139,0.64832,0.21058,0.01627,0.01456
ACH-000121,0,0,0,0.62641,0.05287,0.02185,0.02467,0.01071,0.78717,0.00434,...,0.78417,0.78417,0,NA,0.0115,0.0455,0.98407,0.89893,0.00344,0.00714
ACH-000176,0,0.00588,0.00588,0.84275,0.00112,0.25363,0.28518,0.02155,0.64989,0.00017,...,0.39925,0.39925,0.4576,0.1884,0.0124,0.46105,0.30337,0.95955,0.00409,0.00274
ACH-000677,0,0,0,0.80199,0.38057,0.73054,0.70021,0.00866,0.9027,0.00605,...,0.95334,0.95334,0.0968,NA,0.06667,0,1,1,0.03944,0.04951
ACH-000665,NA,0,0,0.12414,0.00037,0.15559,0.09245,0.0,0.14579,0.00135,...,0.91432,0.91432,0,0,0,0.0096,0.65097,NA,0.02427,0.0126
ACH-000733,0,0.0,0.0,0.11909,0.19022,0.63529,0.72478,0.00704,0.71601,0.02363,...,0.88736,0.88736,0,0,0.0068,0,1,1,0.0,0.0


In [39]:
#drop row that have 'NA' value in the mutation_df
mutation_df.replace('     NA', np.nan, inplace=True)
mutation_df_clean = mutation_df.dropna(axis=1, how='all')
mutation_df_clean.head()

,LOC100288069,LINC01128,LINC01128.1,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,PERM1,HES4,...,VBP1,VBP1.1,SPRY3,TMLHE,SPRY3.1,VAMP7,IL9R,DDX11L16,CD24,CD24.1
Depmap Id,,,,,,,,,,,,,,,,,,,,,
ACH-000766,0.0054,0,0,0.2199,0.00319,0.56622,0.45567,0.15456,0.6955,0.00582,...,0.93662,0.93662,0.0152,0,0.193,0.21055,0.72042,1,0.01729,0.03054
ACH-000638,0.00714,0.00524,0.00524,0.63022,0.68687,0.76282,0.63827,0.00266,0.61781,0.00116,...,0.91177,0.91177,0.9595,0.83335,0.2278,0.07095,0.54917,NaN,0.0238,0.0357
ACH-000563,0.04862,0.05165,0.05165,0.7013,0.03529,0.65608,0.57020,0.15066,0.80167,0.0188,...,1.0,1.0,0.9677,NaN,0.32907,0.0,0.64642,NaN,0.13725,0.18983
ACH-000781,0,0.00303,0.00303,0.55523,0.06030,0.03324,0.02860,0.01503,0.43294,0.0063,...,0.94569,0.94569,0,NaN,0,0.0139,0.64832,0.21058,0.01627,0.01456
ACH-000121,0,0,0,0.62641,0.05287,0.02185,0.02467,0.01071,0.78717,0.00434,...,0.78417,0.78417,0,NaN,0.0115,0.0455,0.98407,0.89893,0.00344,0.00714


In [ ]:
#drop column that have NaN values in the mutation_df
mutation_df_clean = mutation_df_clean.dropna(axis=1, how='any')
mutation_df_clean.head(10)

In [44]:
mutation_df_clean.shape

(50, 8259)

In [45]:
# create df -> tensor to be input for gnn
expression_tensor = np.array(expression_df.values, dtype=np.float32)
mutation_tensor = np.array(mutation_df_clean.values, dtype=np.float32)

In [50]:
# concat omic per cell line 
cell_features = np.concatenate([expression_tensor, mutation_tensor], axis=1)

In [17]:
mutation_df.head()

,Unnamed: 0,cell line name,Depmap Id,LOC100288069,LINC01128,LINC01128.1,LOC100130417,SAMD11,NOC2L,KLHL17,...,VBP1,VBP1.1,SPRY3,TMLHE,SPRY3.1,VAMP7,IL9R,DDX11L16,CD24,CD24.1
0,0,DMS53,ACH-000698,0,0,0,0.24,0.00044,0.01222,0.01244,...,0.98571,0.98571,0,0,0.10967,0.0,0.8609,NA,0.00515,0
1,1,NCIH1184,ACH-000523,0,0,0,0.60134,0.00330,0.18282,0.20615,...,0.93083,0.93083,0,0,0.0485,0.0,0.96913,NA,0.0,0
2,2,NCIH2227,ACH-000610,0,0,0,0.46804,0.00497,0.10173,0.10419,...,0.89152,0.89152,0,NA,0,0.0,0.81503,0.7417,0.02698,0.0214
3,3,RERFLCAD2,ACH-000774,0,0,0,0.81141,0.04441,0.07674,0.07868,...,0.9039,0.9039,0,0,0.09443,0.0,0.75237,NA,0.11971,0.18987
4,4,NCIH2347,ACH-000875,0,0.01477,0.01477,0.10924,0.03631,0.72194,0.51957,...,0.6679,0.6679,0.9,0.47037,0.26403,0.26355,0.4568,0.98017,0.0,0


In [15]:
expression_df.head()

,CCL/Gene,TSPAN6 (7105),TNMD (64102),DPM1 (8813),SCYL3 (57147),C1orf112 (55732),FGR (2268),CFH (3075),FUCA2 (2519),GCLC (2729),...,ZNF772 (400720),ZNF257 (113835),PCNX3 (399909),ADAM32 (203102),ACSL5 (51703),LRRC8B (23507),ABCB8 (11194),H3C12 (8356),SND1 (27044),ZNF785 (146540)
8,ACH-000327,3.337711,0.000000,5.927659,1.944858,2.678072,0.014355,3.090853,6.011451,3.642702,...,2.435629,0.042644,3.727920,0.070389,0.275007,1.903038,4.927896,0.475085,6.916238,2.405992
11,ACH-000705,4.412104,0.000000,6.412782,2.367371,4.275752,0.042644,0.201634,2.563158,6.104546,...,2.381283,0.084064,5.078097,0.970854,0.084064,2.765535,4.148121,0.604071,6.446256,1.321928
14,ACH-000528,4.512227,0.000000,7.099926,2.843984,4.672991,0.014355,0.815575,6.709153,3.982765,...,1.622930,0.000000,4.856488,0.321928,0.176323,3.367371,4.921246,0.378512,6.367546,2.568032
32,ACH-000800,3.135863,0.000000,6.668318,2.592158,4.348374,0.014355,0.422233,5.124742,4.712596,...,2.990955,0.201634,4.495695,0.137504,0.189034,3.978196,4.331992,0.176323,6.510804,3.699330
67,ACH-000176,1.111031,0.042644,7.071677,2.007196,3.586164,0.014355,0.536053,7.026911,4.141596,...,2.100978,2.906891,4.650765,1.063503,2.782409,0.985500,5.785289,0.422233,7.630959,3.212569


In [48]:
#save mutation_df_clean to csv
mutation_df_clean.to_csv("5_dataset/trans_mutation_df_clean.csv", index=True)

In [ ]:
pip install torch

In [49]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
''' ยังไม่ค่อยเข้าใจว่า concat เเบบ encoder คั่น คือทำอย่างไร
def forward(self, gene_expr, mutation):
        gene_expr_emb = self.gene_expr_encoder(gene_expr)
        mutation_emb = self.mutation_encoder(mutation)
        combined = torch.cat([gene_expr_emb, mutation_emb], dim=1)
        fused_emb = self.fusion_layer(combined)
        return fused_emb
'''

2) Drug feature extraction

In [ ]:
pip install rdkit

In [53]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs

In [ ]:
def smiles_to_fingerprint(smiles, n_bits=2048):
    try:
        mol = Chem.MolFromSmiles(smiles) 
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=n_bits)
        arr = np.zeros((n_bits,), dtype=int)
        DataStructs.ConvertToNumpyArray(fp, arr)
        return arr
    except:
        return np.zeros((n_bits,), dtype=int)

# Create a drug with fingerprint mapping (binary fingerprint vector)
drug_fps = {}
for drug, smiles in zip(ic50_df['Drug name'], ic50_df['SELFIES']):
    if drug not in drug_fps:
        drug_fps[drug] = smiles_to_fingerprint(smiles)

In [57]:
drug_fps

{'Camptothecin': array([0, 0, 0, ..., 0, 0, 0], shape=(2048,)),
 'Vinblastine': array([0, 0, 0, ..., 0, 0, 0], shape=(2048,)),
 'Cisplatin': array([0, 0, 0, ..., 0, 0, 0], shape=(2048,)),
 'Cytarabine': array([0, 0, 0, ..., 0, 0, 0], shape=(2048,)),
 'Docetaxel': array([0, 0, 0, ..., 0, 0, 0], shape=(2048,)),
 'Methotrexate': array([0, 0, 0, ..., 0, 0, 0], shape=(2048,)),
 'Tretinoin': array([0, 0, 0, ..., 0, 0, 0], shape=(2048,)),
 'Gefitinib': array([0, 0, 0, ..., 0, 0, 0], shape=(2048,)),
 'Navitoclax': array([0, 0, 0, ..., 0, 0, 0], shape=(2048,)),
 'Vorinostat': array([0, 0, 0, ..., 0, 0, 0], shape=(2048,)),
 'Nilotinib': array([0, 0, 0, ..., 0, 0, 0], shape=(2048,)),
 'Refametinib': array([0, 0, 0, ..., 0, 0, 0], shape=(2048,)),
 'Temsirolimus': array([0, 0, 0, ..., 0, 0, 0], shape=(2048,)),
 'Olaparib': array([0, 0, 0, ..., 0, 0, 0], shape=(2048,)),
 'Veliparib': array([0, 0, 0, ..., 0, 0, 0], shape=(2048,)),
 'Bosutinib': array([0, 0, 0, ..., 0, 0, 0], shape=(2048,)),
 'Lenalid

3) PyG graph input